In [13]:
import gymnasium as gym
import time
env = gym.make("BreakoutNoFrameskip-v4", render_mode='rgb_array')

print("Observation Space: ", env.observation_space)
print("Action Space       ", env.action_space)


obs ,info= env.reset()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 82.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.17.2-py3-none-any.whl size=1650865 sha256=34cd173c8ea35b07c0772faabac2a6b3c2596f97c1be7407fb00a52b94e31bd0
  Stored in directory: /home/simone/.cache/pip/wheels/b5/a2/d3/d3b6cd70bcc1dc95f45b57af0abe0f62f2df65bc5326142a24
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492022 sha256=a1ef2d123a84680281f348ff7346fa73e876b5bb6cf6822f705ca981b9625eb7
  Stored in directory: /home/simone/.cache/pip/wheels/5e/a9/47/f118e66afd12240e4662752cc22cefae5d97275623aa8ef57d
Successfully built gym future
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling clou

In [ ]:
for i in range(100):
    action = env.action_space.sample()
    obs, reward, done,trunc, info = env.step(action)
    #env.render()
    time.sleep(0.01)
env.close()

In [ ]:
from collections import deque
from gymnasium import spaces
import numpy as np

In [ ]:
class ConcatObs(gym.Wrapper):
    def __init__(self,env,k):
        gym.Wrapper.__init__(self,env)
        self.k=k
        self.frames=deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = \
            spaces.Box(low=0,high=255,shape=((k,)+shp),dtype=env.observation_space.dtype)

        

    def reset(self):
        ob,info = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()
    
    def step(self,action):
        ob,reward,done,trunc,info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(),reward,done,trunc,info
    
    def _get_ob(self):
        assert len(self.frames) == self.k
        return np.array(self.frames)

In [ ]:
env = gym.make("BreakoutNoFrameskip-v4")
wrapped_env = ConcatObs(env,4)
print("The new observation space is: ", wrapped_env.observation_space)


In [ ]:
# Reset the Env
obs = wrapped_env.reset()
print("Intial obs is of the shape", obs.shape)

# Take one step
obs, _, _, _,_  = wrapped_env.step(2)
print("Obs after taking a step is", obs.shape)


In [ ]:
import random

from gymnasium.core import Env 

class ObservationWeapper(gym.ObservationWrapper):
    def __init__(self, env: Env):
        super().__init__(env)
        
    def observation(self, observation):
        #normalize 
        return observation/255.0

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env: Env):
        super().__init__(env)
        
    def reward(self, reward):
        #normalize 
        return np.clip(reward, 0, 1)
    
    
    
class ActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def action(self, action):
        if action == 3:
            return random.choice([0,1,2])
        else:
            return action


In [ ]:
env= gym.make("BreakoutNoFrameskip-v4", render_mode='rgb_array')
wrapped_env = ObservationWeapper(RewardWrapper(ActionWrapper(env)))

obs = wrapped_env.reset()

for step in range(500):
    action  = wrapped_env.action_space.sample()
    obs,reward,done,trunc,info = wrapped_env.step(action)
    
    # raise a flaf if value have not been vectorized properly
    if (obs>1.0).any() or (obs<0.0).any():
        raise Exception("Observation not normalized")
    
    
    # raise a flaf if value have not been clipped properly
    if reward>1.0 or reward<0.0:
        raise Exception("Reward not clipped")
    
    # Check in renderring is working
    wrapped_env.render()
    
wrapped_env.close()

print("All tests passed")

In [ ]:
print("Wrapped Env:", wrapped_env)
print("Unwrapped Env", wrapped_env.unwrapped)
print("Getting the meaning of actions", wrapped_env.unwrapped.get_action_meanings())

Use Baselines

In [ ]:

!pip install stable-baselines3[extra]